In [ ]:
import librosa
import librosa.display
import numpy as np
import matplotlib.pyplot as plt
import os
import pandas as pd

folder_path = 'genres_original'

def load_audio_files(folder_path):
    audio_files = []
    for genre in os.listdir(folder_path):
        genre_folder = os.path.join(folder_path, genre)
        if os.path.isdir(genre_folder):
            for file in os.listdir(genre_folder):
                if file.endswith('.wav') and genre_folder == 'genres_original/rock':
                    audio_files.append(os.path.join(genre_folder, file))
    return audio_files





['genres_original/rock/rock.00095.wav', 'genres_original/rock/rock.00022.wav', 'genres_original/rock/rock.00094.wav', 'genres_original/rock/rock.00076.wav', 'genres_original/rock/rock.00012.wav', 'genres_original/rock/rock.00062.wav', 'genres_original/rock/rock.00041.wav', 'genres_original/rock/rock.00046.wav', 'genres_original/rock/rock.00009.wav', 'genres_original/rock/rock.00097.wav', 'genres_original/rock/rock.00078.wav', 'genres_original/rock/rock.00080.wav', 'genres_original/rock/rock.00052.wav', 'genres_original/rock/rock.00011.wav', 'genres_original/rock/rock.00067.wav', 'genres_original/rock/rock.00077.wav', 'genres_original/rock/rock.00084.wav', 'genres_original/rock/rock.00072.wav', 'genres_original/rock/rock.00083.wav', 'genres_original/rock/rock.00042.wav', 'genres_original/rock/rock.00027.wav', 'genres_original/rock/rock.00090.wav', 'genres_original/rock/rock.00056.wav', 'genres_original/rock/rock.00051.wav', 'genres_original/rock/rock.00014.wav', 'genres_original/rock/ro

In [ ]:
import numpy as np
import librosa

def extract_features(file_path):
    y, sr = librosa.load(file_path, sr=None)  # Load the audio file
    
    # Extract MFCCs
    mfccs = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=13)
    mfccs_mean = np.mean(mfccs, axis=1)
    
    # Extract Chroma feature
    chroma = librosa.feature.chroma_stft(y=y, sr=sr)
    chroma_mean = np.mean(chroma, axis=1)
    
    # Spectral Contrast
    spectral_contrast = librosa.feature.spectral_contrast(y=y, sr=sr)
    spectral_contrast_mean = np.mean(spectral_contrast, axis=1)
    
    # Zero Crossing Rate
    zcr = librosa.feature.zero_crossing_rate(y)
    zcr_mean = np.mean(zcr)
    
    # Tempo
    onset_env = librosa.onset.onset_strength(y=y, sr=sr)
    tempo, _ = librosa.beat.beat_track(onset_envelope=onset_env, sr=sr)

    print('Tempo: ' ,tempo)
    print('MFCCs shape:', mfccs_mean)
    

    # Return the features
    return np.concatenate([mfccs_mean, chroma_mean, spectral_contrast_mean, [zcr_mean, tempo]])
    

print(extract_features('genres_original/rock/rock.00083.wav'))

Tempo:  [123.046875]
MFCCs shape: [-119.15098   107.44224   -10.973196   58.555363  -12.933258   26.83433
  -17.646807   19.371737  -15.605269   13.570684  -16.67104    11.870044
  -12.196166]


ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (2,) + inhomogeneous part.

In [ ]:
import os
import pandas as pd


def create_feature_dataset(folder_path):
    audio_files = load_audio_files(folder_path)
    features = []
    for file in audio_files:
        feature_vector = extract_features(file)  # This should be 1D now
        features.append(feature_vector)
    
    # Convert the list of features into a DataFrame
    feature_df = pd.DataFrame(features)
    
    return feature_df

folder_path = 'genres_original'  # Replace with your actual path
features_df = create_feature_dataset(folder_path)



ValueError: all the input arrays must have same number of dimensions, but the array at index 0 has 1 dimension(s) and the array at index 4 has 2 dimension(s)